# Выпускной проект

**Задания**:

- [Посчитайте, сколько книг вышло после 1 января 2000 года;](#1)
- [Для каждой книги посчитайте количество обзоров и среднюю оценку;](#2)
- [Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;](#3)
- [Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;](#4)
- [Посчитайте среднее количество обзоров от пользователей, которые поставили больше 48 оценок;](#5)
- [Выводы.](#6)

# Проект по SQL

**Описание проекта**

Коронавирус застал мир врасплох, изменив привычный порядок вещей. На какое-то время жители городов перестали выходить на улицу, посещать кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать.

Наша компания решила быть на волне и купила крупный сервис для чтения книг по подписке.

**Описание данных**

**Таблица `books`**
Содержит данные о книгах:
- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

**Таблица `authors`**
Содержит данные об авторах:
- `author_id` — идентификатор автора;
- `author` — имя автора.


**Таблица `publishers`**
Содержит данные об издательствах:
- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;


**Таблица `ratings`**
Содержит данные о пользовательских оценках книг:
- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.


**Таблица `reviews`**
Содержит данные о пользовательских обзорах:
- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя автора обзора;
- `text` — текст обзора.

**Цель проекта**

Проанализировать базу данных, сформулировать ценностное предложение для нового продукта.

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

In [2]:
# чтобы выполнить SQL-запрос, используем Pandas
query = '''SELECT * FROM books LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


## Посчитайте, сколько книг вышло после 1 января 2000 года
<a id="1"></a>

In [3]:
query = '''SELECT COUNT(book_id) 
            FROM books 
            WHERE publication_date > '2000-01-01';'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,count
0,819


После 1 января 2000 года вышло 819 книг.

## Для каждой книги посчитайте количество обзоров и среднюю оценку
<a id="2"></a>

In [4]:
query = '''SELECT b.title,
                  b.book_id,
                  COUNT(DISTINCT r.review_id) AS count_review,
                  ROUND(AVG(ra.rating), 2) AS avg_rating
            FROM books AS b
            JOIN reviews AS r ON b.book_id = r.book_id
            JOIN ratings AS ra ON r.book_id = ra.book_id
            GROUP BY b.title, b.book_id
            ORDER BY COUNT(DISTINCT r.review_id) DESC;'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,title,book_id,count_review,avg_rating
0,Twilight (Twilight #1),948,7,3.66
1,The Road,854,6,3.77
2,The Book Thief,656,6,4.26
3,The Glass Castle,734,6,4.21
4,Water for Elephants,963,6,3.98
...,...,...,...,...
989,Naked Empire (Sword of Truth #8),465,1,3.50
990,Moo Baa La La La!,446,1,3.00
991,Merrick (The Vampire Chronicles #7),431,1,4.00
992,Babyville,92,1,3.50


Мы посчитали для каждой книги число обзоров и среднюю оценку. Самое большое количество обзоров получила книга Twilight (Сумерки) - 7, а средняя оценка этой книги - 3.66 На втором месте стоит книга The Road (Дорога) с оценкой 3,77 и 6 отзывами.

In [5]:
query = '''SELECT COUNT(*) AS total_reviews 
           FROM reviews;'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,total_reviews
0,2793


## Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры
<a id="3"></a>

In [6]:
query = '''WITH bok AS (
                    SELECT publisher_id,
                            COUNT(book_id) AS num_books
                    FROM books
                    WHERE num_pages > 50
                    GROUP BY publisher_id)
            SELECT p.publisher,
                    b.num_books
            FROM bok AS b
            JOIN publishers AS p ON b.publisher_id = p.publisher_id
            ORDER BY b.num_books DESC
            LIMIT 1;'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,publisher,num_books
0,Penguin Books,42


Больше всего книг толще 50 страниц выпустило издательство Penguin Books (42 книги).

## Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками
<a id="4"></a>

In [7]:
query = '''WITH rat AS (
                    SELECT book_id,
                           AVG(rating) AS avg_rat
                    FROM ratings
                    GROUP BY book_id
                    HAVING COUNT(rating) >= 50),
                auth AS (
                    SELECT a.author_id,
                            a.author,
                            AVG(r.avg_rat) AS author_avg_rat
                    FROM rat AS r
                    JOIN books AS b ON r.book_id = b.book_id
                    JOIN authors AS a ON b.author_id = a.author_id
                    GROUP BY a.author_id, a.author)
            SELECT author,
                   author_avg_rat
            FROM auth
            ORDER BY author_avg_rat DESC
            LIMIT 1;'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,author,author_avg_rat
0,J.K. Rowling/Mary GrandPré,4.283844


Самую высокую среднюю оценку книг получила автор J.K. Rowling (Джоан Роулинг) - 4.3, чьи книги насчитывают более 50 оценок.

## Посчитайте среднее количество обзоров от пользователей, которые поставили больше 48 оценок
<a id="5"></a>

In [8]:
query = '''WITH rat AS (
                    SELECT username,
                            COUNT(rating_id) AS count_rating
                    FROM ratings
                    GROUP BY username
                    HAVING COUNT(rating_id) > 48),
                rev AS (
                    SELECT r.username,
                            COUNT(r.review_id) AS count_review
                    FROM reviews AS r
                    JOIN rat AS ra ON r.username = ra.username
                    GROUP BY r.username)
            SELECT AVG(count_review) AS avg_count_review,
                    COUNT(*) AS count_rating
            FROM rev;'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,avg_count_review,count_rating
0,24.0,13


Среднее количество обзоров от пользователей, которые поставили больше 48 оценок, составляет 24 обзора. Всего таких пользователей 13.

## Выводы
<a id="6"></a>

В ходе анализа базы данных сервиса для чтения книг, мы пришли к следующим выводам:
1. После 1 января 2000 года вышло 819 книг.
2. Мы посчитали для каждой книги число обзоров и среднюю оценку. Самое большое количество обзоров получила книга Twilight (Сумерки) - 7, а средняя оценка этой книги - 3.66 На втором месте стоит книга The Road (Дорога) с оценкой 3,77 и 6 отзывами.
3. Больше всего книг толще 50 страниц выпустило издательство Penguin Books (42 книги).
4. Самую высокую среднюю оценку книг получила автор J.K. Rowling (Джоан Роулинг) - 4.3, чьи книги насчитывают более 50 оценок. 
5. Среднее количество обзоров от пользователей, которые поставили больше 48 оценок, составляет 24 обзора. Всего таких пользователей 13.